In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import re
import os

from pdfminer.high_level import extract_text, extract_pages
from azure.ai.textanalytics import TextAnalyticsClient, ExtractSummaryAction
from azure.core.credentials import AzureKeyCredential

## Extraction

In [3]:
page_pattern = re.compile(r'Página\s\d+\sde\s\d+')
text_pattern = re.compile(r"1\.\d")
cap_pattern = re.compile(r"\d\.\s+[A-Z]+")

In [4]:
random_numbers = extract_text("/work/pdf/Generar numeros pseudo aleatorios.pdf")
poo_objects = extract_text("/work/pdf/POO Clases y Objetos.pdf")
db = extract_text("/work/pdf/Bases de datos.pdf")

In [5]:
random_numbers = re.split(text_pattern,random_numbers)

In [6]:
# db = [page for array in db for page in array]
random_numbers = random_numbers[-9:]

Due the split was in number.number pattern, at index the document was splitted multiple time, so was necessary remove those splits. From there borns the new caps variable.

## Traslation

Using the traslation functionality to traslate from english output to spanish due model gives the output in english.

## Analysis

Using language services from Azure Cognitive Services from Azure

### Authentication:

In [7]:
key = os.environ["KEY"]
endpoint = os.environ["ENDPOINT"]

In [8]:
text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

### Extractive:

In [9]:
def extractive_summarization(client, doc):

    document = doc
    output = []
    summary = ''

    poller = text_analytics_client.begin_extract_summary(document)
    extract_summary_results = poller.result()
    for result in extract_summary_results:
        if result.kind == "ExtractiveSummarization":
            summary =  " ".join([sentence.text for sentence in result.sentences])
            output.append(summary)
            print("Summaries extractive:")
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))
    
    return output

### Abstractive:

In [10]:
def abstractive_summarization(client,docs:list[str]):

    document:list[str] = docs
    output = []

    poller = text_analytics_client.begin_abstractive_summary(document)
    abstractive_summary_results = poller.result()
    for result in abstractive_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            sumsum = [summary.text for summary in result.summaries]
            output.append(sumsum)
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))    
    
    return output

In [11]:
# db_summary = extractive_summarization(text_analytics_client, db)
# random_summary = abstractive_summarization(text_analytics_client, random_numbers)

In [12]:
# sum_sum = [page for array in random_summary for page in array]

### Persistence:

In [13]:
def persistence(datalist:list[str], name:str) -> None:
    with open(name, mode="a", encoding='utf8') as f:
        for letter in datalist:
            f.write(letter)

In [14]:
# persistence(sum_sum, "random_b.txt")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67cfeb36-b07e-45a6-acf2-83ce38aa6dfe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>